# import modules & load file

## import modules

In [101]:
import pandas as pd
import numpy as np
import math
import glob
import datetime
import os
current_directory = os.getcwd()
print("현재 디렉토리:", current_directory)

현재 디렉토리: d:\myhub\Project1


## load files

In [102]:
folder_path = './option_price/' # 폴더 경로

목표 : 각 옵션종목을 각 dataframe으로 저장
=======
dataframe이름 예로 201TB305는 콜옵션, 23년, 11월, 행사가 305이므로 -> TB305C
201TC2337는 콜옵션, 23년, 12월, 행사가 337.5이므로 -> TC337C
301TB312는 풋옵션, 23년, 11월, 행사가 312.5이므로 -> TB312P
301TC335는 풋옵션, 23년, 12월, 행사가 335이므로 -> TC335P

## CTB0930 콜옵션 23년 11월물 9시30분 현재가

In [103]:
CTB0930 = pd.DataFrame() # 합칠 CSV 파일들을 저장할 빈 DataFrame 생성
root_directory = './option_price/201TB305_352.5/' # 시작 디렉토리 설정

# root_directory와 그 하위 모든 디렉토리에서 CSV 파일을 찾아 합침
for root, dirs, files in os.walk(root_directory):
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(root, file)
            # CSV 파일을 DataFrame으로 읽어들임
            data = pd.read_csv(file_path)
            data['현재가'] = data['현재가'].abs()
            csv_file_name = os.path.basename(file_path)
            if csv_file_name.startswith('201'): # 201로 시작하면 끝에 C를 붙인다.
                csv_file_name = csv_file_name.split('.')[0].split('TB')[1]
            elif csv_file_name.startswith('301'): # 301로 시작하면 끝에 P를 붙인다.
                csv_file_name = csv_file_name.split('.')[0].split('TB')[1]
            data['체결시간'] = pd.to_datetime(data['체결시간'], format="%Y%m%d%H%M%S")
            data = data[['체결시간', '현재가']].rename(columns={'현재가': csv_file_name, '체결시간': 'date'})
            # 읽어들인 데이터를 기존 데이터프레임과 결합
            if CTB0930.empty:
                CTB0930 = data
            else:
                CTB0930 = pd.merge(CTB0930, data, on='date', how='outer')
CTB0930[1:].bfill(inplace=True)
CTB0930 = CTB0930[CTB0930['date'].dt.time == pd.to_datetime('09:30:00').time()] # 09시30분 데이터만 추출
CTB0930['date'] = CTB0930['date'].dt.strftime("%Y-%m-%d") # 'date' 열의 datetime 값을 "YYYY-MM-DD" 형식으로 변경
CTB0930.set_index('date', inplace=True)

C:\Users\jay_hook\AppData\Local\Temp\ipykernel_12908\3322167269.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CTB0930[1:].bfill(inplace=True)


In [104]:
CTB0930.head(11)

,305,307,310,312,315,317,320,322,325,327,...,337,340,342,345,347,350,352,355,357,360
date,,,,,,,,,,,,,,,,,,,,,
2023-10-27,6.45,4.97,3.70,2.65,1.80,1.18,0.73,0.43,0.26,0.15,...,0.04,0.03,0.02,0.01,0.01,0.01,0.01,NaN,NaN,NaN
2023-10-26,8.96,7.16,5.60,4.25,3.11,2.17,1.45,0.92,0.56,0.34,...,0.06,0.05,0.03,0.03,0.02,0.02,0.01,NaN,NaN,NaN
2023-10-25,14.40,11.90,10.00,8.00,6.38,4.91,3.65,2.59,1.76,1.14,...,0.15,0.09,0.06,0.04,0.02,0.02,0.01,NaN,NaN,NaN
2023-10-24,12.60,10.60,8.60,7.00,5.52,4.20,3.09,2.18,1.46,0.94,...,0.12,0.08,0.05,0.04,0.02,0.01,0.01,NaN,NaN,NaN
2023-10-23,14.20,12.30,10.20,8.40,6.81,5.28,4.06,2.95,2.08,1.41,...,0.21,0.12,0.08,0.05,0.03,0.02,0.01,NaN,NaN,NaN
2023-10-20,14.35,12.95,10.55,9.00,7.15,5.71,4.44,3.33,2.44,1.72,...,0.31,0.19,0.12,0.07,0.05,0.02,0.01,NaN,NaN,NaN
2023-10-13,22.80,15.25,13.35,15.40,9.44,11.80,9.63,7.93,6.44,5.04,...,1.50,1.02,0.67,0.42,0.26,0.16,0.10,NaN,NaN,NaN
2023-10-19,NaN,NaN,15.10,13.30,11.30,9.15,7.72,6.14,4.76,3.58,...,0.78,0.48,0.28,0.16,0.09,0.06,0.03,NaN,NaN,NaN
2023-10-18,NaN,NaN,18.75,16.35,14.00,11.85,10.15,8.57,6.87,5.41,...,1.43,0.92,0.56,0.32,0.18,0.10,0.05,NaN,NaN,NaN


In [105]:
CTB0915 = pd.DataFrame() # 합칠 CSV 파일들을 저장할 빈 DataFrame 생성
root_directory = './option_price/201TB305_352.5/' # 시작 디렉토리 설정

# root_directory와 그 하위 모든 디렉토리에서 CSV 파일을 찾아 합침
for root, dirs, files in os.walk(root_directory):
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(root, file)
            # CSV 파일을 DataFrame으로 읽어들임
            data = pd.read_csv(file_path)
            data['현재가'] = data['현재가'].abs()
            csv_file_name = os.path.basename(file_path)
            if csv_file_name.startswith('201'):
                csv_file_name = csv_file_name.split('.')[0].split('TB')[1]
            elif csv_file_name.startswith('301'):
                csv_file_name = csv_file_name.split('.')[0].split('TB')[1]
            data['체결시간'] = pd.to_datetime(data['체결시간'], format="%Y%m%d%H%M%S")
            data = data[['체결시간', '현재가']].rename(columns={'현재가': csv_file_name, '체결시간': 'date'})
            # 읽어들인 데이터를 기존 데이터프레임과 결합
            if CTB0915.empty:
                CTB0915 = data
            else:
                CTB0915 = pd.merge(CTB0915, data, on='date', how='outer')
CTB0915[1:].bfill(inplace=True)
CTB0915 = CTB0915[CTB0915['date'].dt.time == pd.to_datetime('09:15:00').time()] # 09시30분 데이터만 추출
CTB0915['date'] = CTB0915['date'].dt.strftime("%Y-%m-%d") # 'date' 열의 datetime 값을 "YYYY-MM-DD" 형식으로 변경
CTB0915.set_index('date', inplace=True)

C:\Users\jay_hook\AppData\Local\Temp\ipykernel_12908\2642951010.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CTB0915[1:].bfill(inplace=True)


In [106]:
CTB0915.head(11)

,305,307,310,312,315,317,320,322,325,327,...,337,340,342,345,347,350,352,355,357,360
date,,,,,,,,,,,,,,,,,,,,,
2023-10-27,6.74,5.21,3.90,2.81,1.93,1.27,0.79,0.48,0.28,0.17,...,0.04,0.03,0.02,0.01,0.01,0.01,0.01,NaN,NaN,NaN
2023-10-26,9.14,7.40,5.79,4.39,3.22,2.25,1.51,0.97,0.60,0.36,...,0.06,0.04,0.03,0.02,0.02,0.01,0.01,NaN,NaN,NaN
2023-10-25,15.45,12.95,10.50,9.14,6.94,5.31,4.00,2.87,1.98,1.30,...,0.17,0.10,0.07,0.04,0.02,0.02,0.01,NaN,NaN,NaN
2023-10-24,13.25,11.00,9.09,7.33,5.77,4.43,3.28,2.32,1.57,1.01,...,0.13,0.08,0.06,0.04,0.02,0.02,0.01,NaN,NaN,NaN
2023-10-20,14.95,12.95,10.95,9.31,7.55,6.04,4.70,3.59,2.64,1.86,...,0.32,0.19,0.12,0.07,0.05,0.03,0.01,NaN,NaN,NaN
2023-10-19,NaN,17.35,15.20,13.05,11.15,9.09,7.50,6.03,4.65,3.49,...,0.75,0.46,0.27,0.15,0.08,0.04,0.03,NaN,NaN,NaN
2023-10-23,NaN,NaN,9.65,8.15,6.38,4.97,3.78,2.76,1.93,1.30,...,0.19,0.11,0.07,0.05,0.03,0.02,0.01,NaN,NaN,NaN
2023-10-18,NaN,NaN,18.00,16.35,13.70,11.85,10.30,8.14,6.52,5.07,...,1.30,0.83,0.50,0.29,0.17,0.10,0.05,NaN,NaN,NaN
2023-10-13,NaN,NaN,17.90,15.40,14.00,11.50,9.70,7.95,6.36,5.03,...,1.47,1.00,0.66,0.42,0.26,0.17,0.10,NaN,NaN,NaN


## CTB1515 콜 23년 11월 15시15분 현재가

In [107]:
CTB1515 = pd.DataFrame() # 합칠 CSV 파일들을 저장할 빈 DataFrame 생성
root_directory = './option_price/201TB305_352.5/' # 시작 디렉토리 설정

# root_directory와 그 하위 모든 디렉토리에서 CSV 파일을 찾아 합침
for root, dirs, files in os.walk(root_directory):
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(root, file)
            # CSV 파일을 DataFrame으로 읽어들임
            data = pd.read_csv(file_path)
            data['현재가'] = data['현재가'].abs()
            csv_file_name = os.path.basename(file_path)
            if csv_file_name.startswith('201'): # 201로 시작하면 끝에 C를 붙인다.
                csv_file_name = csv_file_name.split('.')[0].split('TB')[1]
            elif csv_file_name.startswith('301'): # 301로 시작하면 끝에 P를 붙인다.
                csv_file_name = csv_file_name.split('.')[0].split('TB')[1]
            data['체결시간'] = pd.to_datetime(data['체결시간'], format="%Y%m%d%H%M%S") # 타입 변경
            data = data[['체결시간', '현재가']].rename(columns={'현재가': csv_file_name, # 행사가를 컬럼 이름으로 넣는다.          
             '체결시간': 'date'}) # 컬럼 이름을 date로 바꾼다.
            # 읽어들인 데이터를 기존 데이터프레임과 결합
            if CTB1515.empty:
                CTB1515 = data
            else:
                CTB1515 = pd.merge(CTB1515, data, on='date', how='outer') # date기준으로 병합
CTB1515[1:].bfill(inplace=True)            
CTB1515 = CTB1515[CTB1515['date'].dt.time == pd.to_datetime('15:15:00').time()] # 15시15분 데이터만 추출         
CTB1515['date'] = CTB1515['date'].dt.strftime("%Y-%m-%d") # 'date' 열의 datetime 값을 "YYYY-MM-DD" 형식으로 변경
CTB1515.set_index('date', inplace=True)

C:\Users\jay_hook\AppData\Local\Temp\ipykernel_12908\2684465963.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CTB1515[1:].bfill(inplace=True)


In [108]:
CTB1515.head(11)

,305,307,310,312,315,317,320,322,325,327,...,337,340,342,345,347,350,352,355,357,360
date,,,,,,,,,,,,,,,,,,,,,
2023-10-27,6.65,5.15,3.82,2.72,1.86,1.20,0.73,0.43,0.25,0.15,...,0.02,0.02,0.02,0.01,0.01,0.01,0.01,NaN,NaN,NaN
2023-10-26,7.15,5.64,4.29,3.15,2.23,1.52,0.99,0.63,0.39,0.24,...,0.05,0.03,0.03,0.02,0.01,0.01,0.01,NaN,NaN,NaN
2023-10-25,13.15,11.00,9.08,7.30,5.68,4.29,3.12,2.16,1.43,0.89,...,0.11,0.06,0.05,0.03,0.02,0.02,0.01,NaN,NaN,NaN
2023-10-23,13.00,10.85,9.00,7.39,5.77,4.44,3.30,2.36,1.62,1.06,...,0.15,0.09,0.06,0.04,0.02,0.02,0.01,NaN,NaN,NaN
2023-10-24,NaN,12.60,10.60,8.66,7.11,5.58,4.25,3.10,2.17,1.45,...,0.19,0.12,0.07,0.05,0.03,0.02,0.01,NaN,NaN,NaN
2023-10-20,NaN,12.80,10.75,9.17,7.40,5.89,4.58,3.46,2.52,1.77,...,0.29,0.19,0.12,0.07,0.04,0.03,0.01,NaN,NaN,NaN
2023-10-17,NaN,NaN,18.55,16.35,14.20,12.10,10.20,8.31,6.68,5.21,...,1.35,0.87,0.52,0.30,0.18,0.10,0.06,NaN,NaN,NaN
2023-10-13,NaN,NaN,17.55,15.90,13.35,11.25,9.33,7.68,6.15,4.81,...,1.36,0.90,0.58,0.36,0.22,0.13,0.08,NaN,NaN,NaN
2023-10-06,NaN,NaN,12.80,10.65,9.25,7.43,6.15,4.93,3.82,2.94,...,0.82,0.57,0.39,0.28,0.18,0.13,0.10,NaN,NaN,NaN


## CTC0930 콜 23년 12월물 9시30분 현재가

In [109]:
CTC0930 = pd.DataFrame() # 합칠 CSV 파일들을 저장할 빈 DataFrame 생성
root_directory = './option_price/201TC305_337.5/' # 시작 디렉토리 설정

# root_directory와 그 하위 모든 디렉토리에서 CSV 파일을 찾아 합침
for root, dirs, files in os.walk(root_directory):
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(root, file)
            # CSV 파일을 DataFrame으로 읽어들임
            data = pd.read_csv(file_path)
            data['현재가'] = data['현재가'].abs()
            csv_file_name = os.path.basename(file_path)
            if csv_file_name.startswith('201'): # 201로 시작하면 끝에 C를 붙인다.
                csv_file_name = csv_file_name.split('.')[0].split('TC')[1]
            elif csv_file_name.startswith('301'): # 301로 시작하면 끝에 P를 붙인다.
                csv_file_name = csv_file_name.split('.')[0].split('TC')[1]
            data['체결시간'] = pd.to_datetime(data['체결시간'], format="%Y%m%d%H%M%S")
            data = data[['체결시간', '현재가']].rename(columns={'현재가': csv_file_name, '체결시간': 'date'})
            data.bfill(inplace=True)            
            selected_rows = data[data['date'].dt.time == pd.to_datetime('09:30:00').time()]
            # print(selected_rows)
            # 읽어들인 데이터를 기존 데이터프레임과 결합
            if CTC0930.empty:
                CTC0930 = data
            else:
                CTC0930 = pd.merge(CTC0930, data, on='date', how='outer') # date기준으로 병합
CTC0930[1:].bfill(inplace=True)        
CTC0930 = CTC0930[CTC0930['date'].dt.time == pd.to_datetime('09:30:00').time()] # 09시30분 데이터만 추출  
CTC0930['date'] = CTC0930['date'].dt.strftime("%Y-%m-%d") # 'date' 열의 datetime 값을 "YYYY-MM-DD" 형식으로 변경
CTC0930.set_index('date', inplace=True)

C:\Users\jay_hook\AppData\Local\Temp\ipykernel_12908\3229019414.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CTC0930[1:].bfill(inplace=True)


In [110]:
CTC0930.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37 entries, 2023-10-27 to 2023-06-05
Data columns (total 22 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   305     3 non-null      float64
 1   307     3 non-null      float64
 2   310     6 non-null      float64
 3   312     7 non-null      float64
 4   315     10 non-null     float64
 5   317     14 non-null     float64
 6   320     15 non-null     float64
 7   322     17 non-null     float64
 8   325     20 non-null     float64
 9   327     21 non-null     float64
 10  330     31 non-null     float64
 11  332     31 non-null     float64
 12  335     37 non-null     float64
 13  337     0 non-null      float64
 14  340     0 non-null      float64
 15  342     0 non-null      float64
 16  345     0 non-null      float64
 17  347     0 non-null      float64
 18  350     0 non-null      float64
 19  352     0 non-null      float64
 20  355     0 non-null      float64
 21  360     0 non-null      float

## CTC1515 콜옵션 23년 12월물 15시15분 현재가

In [111]:
CTC1515 = pd.DataFrame() # 합칠 CSV 파일들을 저장할 빈 DataFrame 생성
root_directory = './option_price/201TC305_337.5/' # 시작 디렉토리 설정

# root_directory와 그 하위 모든 디렉토리에서 CSV 파일을 찾아 합침
for root, dirs, files in os.walk(root_directory):
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(root, file)
            # CSV 파일을 DataFrame으로 읽어들임
            data = pd.read_csv(file_path)
            data['현재가'] = data['현재가'].abs()
            csv_file_name = os.path.basename(file_path)
            if csv_file_name.startswith('201'): # 201로 시작하면 끝에 C를 붙인다.
                csv_file_name = csv_file_name.split('.')[0].split('TC')[1]
            elif csv_file_name.startswith('301'): # 301로 시작하면 끝에 P를 붙인다.
                csv_file_name = csv_file_name.split('.')[0].split('TC')[1]
            data['체결시간'] = pd.to_datetime(data['체결시간'], format="%Y%m%d%H%M%S")
            data = data[['체결시간', '현재가']].rename(columns={'현재가': csv_file_name, '체결시간': 'date'})            
            # 읽어들인 데이터를 기존 데이터프레임과 결합
            if CTC1515.empty:
                CTC1515 = data
            else:
                CTC1515 = pd.merge(CTC1515, data, on='date', how='outer') # date기준으로 병합
CTC1515[1:].bfill(inplace=True)
CTC1515 = CTC1515[CTC1515['date'].dt.time == pd.to_datetime('15:15:00').time()] # 09시30분 데이터만 추출  
CTC1515['date'] = CTC1515['date'].dt.strftime("%Y-%m-%d") # 'date' 열의 datetime 값을 "YYYY-MM-DD" 형식으로 변경
CTC1515.set_index('date', inplace=True)

C:\Users\jay_hook\AppData\Local\Temp\ipykernel_12908\95526729.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CTC1515[1:].bfill(inplace=True)


In [112]:
CTC1515.head()

,305,307,310,312,315,317,320,322,325,327,...,335,337,340,342,345,347,350,352,355,360
date,,,,,,,,,,,,,,,,,,,,,
2023-10-26,11.25,9.80,8.49,7.16,6.00,5.00,4.09,3.30,2.62,2.08,...,0.95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-10-20,17.90,16.30,14.30,18.60,11.05,9.67,8.25,7.06,5.94,4.94,...,2.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-09-07,33.80,9.43,29.35,6.72,5.61,4.54,3.68,2.95,2.33,1.83,...,10.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-10-27,NaN,9.43,7.95,6.72,5.61,4.54,3.68,2.95,2.33,1.83,...,0.77,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-10-23,NaN,NaN,12.90,11.10,9.79,8.45,6.93,5.98,4.95,4.01,...,2.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## PTB0930 풋옵션 23년 11월물 9시30분 현재가

In [113]:
PTB0930 = pd.DataFrame() # 합칠 CSV 파일들을 저장할 빈 DataFrame 생성
root_directory = './option_price/301TB305_352.5/' # 시작 디렉토리 설정

# root_directory와 그 하위 모든 디렉토리에서 CSV 파일을 찾아 합침
for root, dirs, files in os.walk(root_directory):
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(root, file)
            # CSV 파일을 DataFrame으로 읽어들임
            data = pd.read_csv(file_path)
            data['현재가'] = data['현재가'].abs()
            csv_file_name = os.path.basename(file_path)
            if csv_file_name.startswith('201'): # 201로 시작하면 끝에 C를 붙인다.
                csv_file_name = csv_file_name.split('.')[0].split('TB')[1]
            elif csv_file_name.startswith('301'): # 301로 시작하면 끝에 P를 붙인다.
                csv_file_name = csv_file_name.split('.')[0].split('TB')[1]
            data['체결시간'] = pd.to_datetime(data['체결시간'], format="%Y%m%d%H%M%S")
            data = data[['체결시간', '현재가']].rename(columns={'현재가': csv_file_name, '체결시간': 'date'})
            
            # 읽어들인 데이터를 기존 데이터프레임과 결합
            if PTB0930.empty:
                PTB0930 = data
            else:
                PTB0930 = pd.merge(PTB0930, data, on='date', how='outer') # date기준으로 병합
PTB0930[1:].bfill(inplace=True)            
PTB0930 = PTB0930[PTB0930['date'].dt.time == pd.to_datetime('09:30:00').time()] # 15시15분 데이터만 추출  
PTB0930['date'] = PTB0930['date'].dt.strftime("%Y-%m-%d") # 'date' 열의 datetime 값을 "YYYY-MM-DD" 형식으로 변경
PTB0930.set_index('date', inplace=True)

C:\Users\jay_hook\AppData\Local\Temp\ipykernel_12908\3713745481.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PTB0930[1:].bfill(inplace=True)


In [114]:
PTB0930.head(11)

,305,307,310,312,315,317,320,322,325,327,...,337,340,342,345,347,350,352,355,357,360
date,,,,,,,,,,,,,,,,,,,,,
2023-10-27,3.42,4.42,5.64,7.06,8.70,10.65,12.50,15.05,17.60,18.35,...,27.50,30.30,33.40,35.00,37.60,39.95,35.05,NaN,NaN,NaN
2023-10-26,2.43,3.18,4.11,5.23,6.49,8.02,9.85,11.80,14.20,16.30,...,25.95,27.40,25.10,26.45,28.95,32.50,35.05,NaN,NaN,NaN
2023-10-25,1.05,1.44,1.95,2.61,3.44,4.45,5.69,7.03,8.88,9.47,...,18.85,21.30,23.80,26.45,28.95,31.30,37.40,NaN,NaN,NaN
2023-10-24,1.63,2.14,2.78,3.57,4.55,5.72,7.14,8.80,10.50,12.50,...,21.45,23.50,26.80,27.45,19.10,33.55,36.15,NaN,NaN,NaN
2023-10-23,1.61,2.07,2.65,3.35,4.22,5.25,6.43,7.85,9.56,10.90,...,20.00,22.20,23.65,27.45,19.10,30.80,23.40,NaN,NaN,NaN
2023-10-20,2.00,2.50,3.12,3.85,4.75,5.83,6.99,8.38,10.00,11.85,...,19.60,22.50,24.90,25.50,19.10,32.40,23.40,NaN,NaN,NaN
2023-10-19,0.71,0.93,1.23,1.61,2.09,2.70,3.46,4.38,5.50,6.82,...,14.35,16.10,18.70,19.70,19.10,25.80,23.40,NaN,NaN,NaN
2023-10-18,0.34,0.46,0.62,0.83,1.13,1.52,2.03,2.66,3.47,4.48,...,11.20,12.75,15.05,17.00,20.40,21.50,25.00,NaN,NaN,NaN
2023-10-17,0.50,0.68,0.93,1.25,1.65,2.17,2.83,3.64,4.63,5.83,...,12.00,14.65,16.50,18.60,25.35,24.15,25.65,NaN,NaN,NaN


## PTB1515 풋옵션 23년 11월물 15시15분 현재가

In [115]:
PTB1515 = pd.DataFrame() # 합칠 CSV 파일들을 저장할 빈 DataFrame 생성
root_directory = './option_price/301TB305_352.5/' # 시작 디렉토리 설정

# root_directory와 그 하위 모든 디렉토리에서 CSV 파일을 찾아 합침
for root, dirs, files in os.walk(root_directory):
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(root, file)
            # CSV 파일을 DataFrame으로 읽어들임
            data = pd.read_csv(file_path)
            data['현재가'] = data['현재가'].abs()
            csv_file_name = os.path.basename(file_path)
            if csv_file_name.startswith('201'): # 201로 시작하면 끝에 C를 붙인다.
                csv_file_name = csv_file_name.split('.')[0].split('TB')[1]
            elif csv_file_name.startswith('301'): # 301로 시작하면 끝에 P를 붙인다.
                csv_file_name = csv_file_name.split('.')[0].split('TB')[1]
            data['체결시간'] = pd.to_datetime(data['체결시간'], format="%Y%m%d%H%M%S")
            data = data[['체결시간', '현재가']].rename(columns={'현재가': csv_file_name, '체결시간': 'date'})            
            # 읽어들인 데이터를 기존 데이터프레임과 결합
            if PTB1515.empty:
                PTB1515 = data
            else:
                PTB1515 = pd.merge(PTB1515, data, on='date', how='outer') # date기준으로 병합
PTB1515[1:].bfill(inplace=True)
PTB1515 = PTB1515[PTB1515['date'].dt.time == pd.to_datetime('15:15:00').time()] # 15시15분 데이터만 추출
PTB1515['date'] = PTB1515['date'].dt.strftime("%Y-%m-%d") # 'date' 열의 datetime 값을 "YYYY-MM-DD" 형식으로 변경
PTB1515.set_index('date', inplace=True)

C:\Users\jay_hook\AppData\Local\Temp\ipykernel_12908\3729456411.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PTB1515[1:].bfill(inplace=True)


In [116]:
PTB1515.head()

,305,307,310,312,315,317,320,322,325,327,...,337,340,342,345,347,350,352,355,357,360
date,,,,,,,,,,,,,,,,,,,,,
2023-10-27,3.04,3.99,5.14,6.56,8.23,10.05,12.10,14.30,16.60,18.55,...,28.85,31.60,33.4,35.00,39.05,40.40,43.60,NaN,NaN,NaN
2023-10-26,3.56,4.52,5.69,7.04,8.63,10.35,12.25,14.35,16.75,19.05,...,28.00,31.05,25.1,26.45,37.60,40.20,35.05,NaN,NaN,NaN
2023-10-25,1.13,1.55,2.10,2.81,3.70,4.79,6.11,7.63,9.48,11.40,...,20.15,22.00,25.1,26.45,28.95,32.50,35.05,NaN,NaN,NaN
2023-10-24,1.08,1.45,1.93,2.55,3.33,4.28,5.42,6.84,8.52,10.20,...,19.80,21.30,26.8,26.45,28.95,33.00,37.40,NaN,NaN,NaN
2023-10-23,1.78,2.30,2.95,3.74,4.70,5.85,7.24,8.78,10.50,12.40,...,21.45,23.80,26.8,27.45,19.10,33.55,36.15,NaN,NaN,NaN


## PTC0930 풋 23년 12월물 9시30분 현재가

In [117]:
PTC0930 = pd.DataFrame() # 합칠 CSV 파일들을 저장할 빈 DataFrame 생성
root_directory = './option_price/301TC305_337.5/' # 시작 디렉토리 설정

# root_directory와 그 하위 모든 디렉토리에서 CSV 파일을 찾아 합침
for root, dirs, files in os.walk(root_directory):
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(root, file)
            # CSV 파일을 DataFrame으로 읽어들임
            data = pd.read_csv(file_path)
            data['현재가'] = data['현재가'].abs()
            csv_file_name = os.path.basename(file_path)
            if csv_file_name.startswith('201'): # 201로 시작하면 끝에 C를 붙인다.
                csv_file_name = csv_file_name.split('.')[0].split('TC')[1]
            elif csv_file_name.startswith('301'): # 301로 시작하면 끝에 P를 붙인다.
                csv_file_name = csv_file_name.split('.')[0].split('TC')[1]
            data['체결시간'] = pd.to_datetime(data['체결시간'], format="%Y%m%d%H%M%S")
            data = data[['체결시간', '현재가']].rename(columns={'현재가': csv_file_name, '체결시간': 'date'})
             # 읽어들인 데이터를 기존 데이터프레임과 결합
            if PTC0930.empty:
                PTC0930 = data
            else:
                PTC0930 = pd.merge(PTC0930, data, on='date', how='outer') # date기준으로 병합
PTC0930[1:].bfill(inplace=True)
PTC0930 = PTC0930[PTC0930['date'].dt.time == pd.to_datetime('09:30:00').time()] # 09시30분 데이터만 추출
PTC0930['date'] = PTC0930['date'].dt.strftime("%Y-%m-%d") # 'date' 열의 datetime 값을 "YYYY-MM-DD" 형식으로 변경
PTC0930.set_index('date', inplace=True)    

C:\Users\jay_hook\AppData\Local\Temp\ipykernel_12908\84916964.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PTC0930[1:].bfill(inplace=True)


In [118]:
PTC0930.head()

,305,307,310,312,315,317,320,322,325,327,...,337,340,342,345,347,350,352,355,357,360
date,,,,,,,,,,,,,,,,,,,,,
2023-10-27,6.62,7.66,8.77,9.99,11.45,12.20,14.50,16.10,16.65,13.15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-10-26,5.61,6.41,7.39,8.66,10.00,10.65,12.55,13.75,15.30,13.15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-10-25,3.54,4.12,4.80,5.52,6.50,7.30,8.45,9.80,10.40,13.15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-10-24,4.15,4.79,5.72,6.30,7.65,8.70,9.94,11.20,12.60,13.05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-10-23,3.94,4.62,5.30,6.13,7.06,8.20,9.45,10.40,12.40,13.30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## PTC1515 풋 23년 12월물 15시15분 현재가

In [119]:
PTC1515 = pd.DataFrame() # 합칠 CSV 파일들을 저장할 빈 DataFrame 생성
root_directory = './option_price/301TC305_337.5/' # 시작 디렉토리 설정

# root_directory와 그 하위 모든 디렉토리에서 CSV 파일을 찾아 합침
for root, dirs, files in os.walk(root_directory):
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(root, file)
            # CSV 파일을 DataFrame으로 읽어들임
            data = pd.read_csv(file_path)
            data['현재가'] = data['현재가'].abs()
            csv_file_name = os.path.basename(file_path)
            if csv_file_name.startswith('201'): # 201로 시작하면 끝에 C를 붙인다.
                csv_file_name = csv_file_name.split('.')[0].split('TC')[1]
            elif csv_file_name.startswith('301'): # 301로 시작하면 끝에 P를 붙인다.
                csv_file_name = csv_file_name.split('.')[0].split('TC')[1]
            data['체결시간'] = pd.to_datetime(data['체결시간'], format="%Y%m%d%H%M%S")
            data = data[['체결시간', '현재가']].rename(columns={'현재가': csv_file_name, '체결시간': 'date'})
            
             # 읽어들인 데이터를 기존 데이터프레임과 결합
            if PTC1515.empty:
                PTC1515 = data
            else:
                PTC1515 = pd.merge(PTC1515, data, on='date', how='outer') # date기준으로 병합
PTC1515[1:].bfill(inplace=True)
PTC1515 = PTC1515[PTC1515['date'].dt.time == pd.to_datetime('15:15:00').time()] # 15시15분 데이터만 추출
PTC1515['date'] = PTC1515['date'].dt.strftime("%Y-%m-%d") # 'date' 열의 datetime 값을 "YYYY-MM-DD" 형식으로 변경
PTC1515.set_index('date', inplace=True)

C:\Users\jay_hook\AppData\Local\Temp\ipykernel_12908\707826619.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PTC1515[1:].bfill(inplace=True)


In [120]:
PTC1515.head()

,305,307,310,312,315,317,320,322,325,327,...,337,340,342,345,347,350,352,355,357,360
date,,,,,,,,,,,,,,,,,,,,,
2023-10-27,6.38,7.30,8.49,9.56,11.20,12.70,14.10,15.90,16.40,18.45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-10-26,6.77,7.74,8.92,10.05,11.55,12.00,14.40,14.95,17.60,13.15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-10-25,3.75,4.31,5.11,5.90,6.87,7.86,9.07,9.98,11.85,13.15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-10-24,3.56,4.15,4.89,5.57,6.45,7.42,8.55,9.76,11.20,13.15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-10-23,4.33,5.04,5.78,6.88,7.82,8.70,10.30,11.20,12.95,13.90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 거래일 기간 조정

In [121]:
def find_monday_of_week_containing_date(target_date):
    return target_date - datetime.timedelta(days=target_date.weekday())

def find_second_thursday(year, month):
    current_date = datetime.date(year, month, 1)
    current_date += datetime.timedelta(days=(3 - current_date.weekday() + 7) % 7)
    current_date += datetime.timedelta(weeks=1)
    return current_date

year, month = 2023, 11  # 연도와 월 설정
date_format = "%Y-%m-%d" # 날짜를 "YYYY-MM-DD" 형식으로 출력
current_month_second_thursday = find_second_thursday(year, month) # 현재 월의 두 번째 목요일
current_month_monday_of_week = find_monday_of_week_containing_date(current_month_second_thursday) # 그 주의 월요일
previous_month = month - 1 if month > 1 else 12
previous_year = year - 1 if month == 1 else year
previous_month_second_thursday = find_second_thursday(previous_year, previous_month) # 이전 월의 두 번째 목요일
previous_month_monday_of_week = find_monday_of_week_containing_date(previous_month_second_thursday) # 그 주의 월요일
current_month_monday_of_week = current_month_monday_of_week.strftime(date_format)
previous_month_monday_of_week =previous_month_monday_of_week.strftime(date_format)
print(previous_month_monday_of_week, type(previous_month_monday_of_week))

2023-10-09 <class 'str'>


# strike price set

In [122]:
strike_price = pd.read_csv('./old_dataset/strike_price.csv')
strike_price['일자'] = pd.to_datetime(strike_price['일자'], format='%Y%m%d').dt.strftime("%Y-%m-%d")
strike_price.set_index('일자', inplace=True)
print(strike_price)

            기준행사가    S-4    S-3    S-2    S-1    S+1    S+2    S+3    S+4
일자                                                                       
2023-10-20  322.5  312.5  315.0  317.5  320.0  325.0  327.5  330.0  332.5
2023-10-19  327.5  317.5  320.0  322.5  325.0  330.0  332.5  335.0  337.5
2023-10-18  325.0  315.0  317.5  320.0  322.5  327.5  330.0  332.5  335.0
2023-10-17  322.5  312.5  315.0  317.5  320.0  325.0  327.5  330.0  332.5
2023-10-16  325.0  315.0  317.5  320.0  322.5  327.5  330.0  332.5  335.0
2023-10-13  327.5  317.5  320.0  322.5  325.0  330.0  332.5  335.0  337.5
2023-10-12  322.5  312.5  315.0  317.5  320.0  325.0  327.5  330.0  332.5
2023-10-11  317.5  307.5  310.0  312.5  315.0  320.0  322.5  325.0  327.5
2023-10-10  317.5  307.5  310.0  312.5  315.0  320.0  322.5  325.0  327.5
2023-10-06  317.5  307.5  310.0  312.5  315.0  320.0  322.5  325.0  327.5
2023-10-05  317.5  307.5  310.0  312.5  315.0  320.0  322.5  325.0  327.5
2023-10-04  325.0  315.0  317.5  320.0

In [123]:
CTB9sp = strike_price.copy()
CTB15sp = strike_price.copy()
PTB9sp = strike_price.copy()
PTB15sp = strike_price.copy()

In [124]:
for c in strike_price.columns:
    for i in strike_price.index:
        value = strike_price.loc[i, c]
        if not np.isnan(value):
            try:
                lookup_value = CTB0930.loc[i, str(math.floor(value))]
                CTB9sp.loc[i, c] = lookup_value
            except KeyError:
                CTB9sp.loc[i, c] = np.nan
        else:
            CTB9sp.loc[i, c] = np.nan
CTB9sp = CTB9sp.loc[(CTB9sp.index >= previous_month_monday_of_week) & (CTB9sp.index < current_month_monday_of_week),:]
print(CTB9sp)

            기준행사가    S-4    S-3    S-2    S-1   S+1   S+2   S+3   S+4
일자                                                                   
2023-10-20   3.33   9.00   7.15   5.71   4.44  2.44  1.72  1.17  0.77
2023-10-19   3.58   9.15   7.72   6.14   4.76  2.60  1.80  1.22  0.78
2023-10-18   6.87  14.00  11.85  10.15   8.57  5.41  4.10  3.00  2.12
2023-10-17   7.17  15.25  13.80  10.70   8.90  5.59  4.30  3.20  2.30
2023-10-16   5.08  12.25  10.40   8.05   6.47  3.90  2.86  2.03  1.39
2023-10-13   5.04  11.80   9.63   7.93   6.44  3.91  2.92  2.13  1.50
2023-10-12   9.33  16.00  15.20  13.15  11.00  7.66  6.22  4.89  3.75
2023-10-11  11.45    NaN  17.45  11.10  13.15  9.60  7.95  6.50  5.17
2023-10-10  10.75    NaN  16.75  10.80  12.75  8.95  7.40  6.00  4.72


In [125]:
for c in strike_price.columns:
    for i in strike_price.index:
        value = strike_price.loc[i, c]
        if not np.isnan(value):
            try:
                lookup_value = CTB1515.loc[i, str(math.floor(value))]
                CTB15sp.loc[i, c] = lookup_value
            except KeyError:
                CTB15sp.loc[i, c] = np.nan
        else:
            CTB15sp.loc[i, c] = np.nan
CTB15sp = CTB15sp.loc[(CTB15sp.index >= previous_month_monday_of_week) & (CTB15sp.index < current_month_monday_of_week),:]
print(CTB15sp)

            기준행사가    S-4    S-3    S-2    S-1   S+1   S+2   S+3   S+4
일자                                                                   
2023-10-20   3.46   9.17   7.40   5.89   4.58  2.52  1.77  1.20  0.78
2023-10-19   3.28   8.90   7.20   5.70   4.40  2.37  1.64  1.08  0.69
2023-10-18   7.40    NaN    NaN  11.10   9.15  5.80  4.43  3.26  2.32
2023-10-17   8.31  16.35  14.20  12.10  10.20  6.68  5.21  3.95  2.87
2023-10-16   4.86  11.55   9.65   7.78   6.30  3.70  2.71  1.92  1.30
2023-10-13   4.81  11.25   9.33   7.68   6.15  3.71  2.74  1.97  1.36
2023-10-12   9.81    NaN    NaN  13.80  11.65  8.15  6.54  5.21  4.03
2023-10-11    NaN    NaN    NaN    NaN    NaN  9.19  7.46  6.02  4.75
2023-10-10   7.68    NaN    NaN  11.05   9.30  6.37  5.00  3.90  2.96


In [126]:
for c in strike_price.columns:
    for i in strike_price.index:
        value = strike_price.loc[i, c]
        if not np.isnan(value):
            try:
                lookup_value = PTB0930.loc[i, str(math.floor(value))]
                PTB9sp.loc[i, c] = lookup_value
            except KeyError:
                PTB9sp.loc[i, c] = np.nan
        else:
            PTB9sp.loc[i, c] = np.nan
PTB9sp = PTB9sp.loc[(PTB9sp.index >= previous_month_monday_of_week) & (PTB9sp.index < current_month_monday_of_week),:]
print(PTB9sp)

            기준행사가   S-4   S-3   S-2   S-1    S+1    S+2    S+3    S+4
일자                                                                   
2023-10-20   8.38  3.85  4.75  5.83  6.99  10.00  11.85  13.80  15.70
2023-10-19   6.82  2.70  3.46  4.38  5.50   8.35  10.00  12.00  14.35
2023-10-18   3.47  1.13  1.52  2.03  2.66   4.48   5.65   7.08   8.70
2023-10-17   3.64  1.25  1.65  2.17  2.83   4.63   5.83   7.23   8.73
2023-10-16   5.58  2.20  2.82  3.56  4.50   6.88   8.33   9.80  11.75
2023-10-13   5.71  2.22  2.86  3.63  4.63   7.10   8.56  10.60  12.20
2023-10-12   3.19  1.17  1.52  1.94  2.51   4.00   5.01   6.21   7.60
2023-10-11   2.55  0.95  1.22  1.59  2.01   3.23   4.04   5.04   6.22
2023-10-10   2.99  1.17  1.50  1.90  2.42   3.75   4.67   5.71   6.95


In [127]:
for c in strike_price.columns:
    for i in strike_price.index:
        value = strike_price.loc[i, c]
        if not np.isnan(value):
            try:
                lookup_value = PTB1515.loc[i, str(math.floor(value))]
                PTB15sp.loc[i, c] = lookup_value
            except KeyError:
                PTB15sp.loc[i, c] = np.nan
        else:
            PTB15sp.loc[i, c] = np.nan
PTB15sp = PTB15sp.loc[(PTB15sp.index >= previous_month_monday_of_week) & (PTB15sp.index < current_month_monday_of_week),:]
print(PTB15sp)

            기준행사가   S-4   S-3   S-2   S-1   S+1    S+2    S+3    S+4
일자                                                                  
2023-10-20   7.89  3.47  4.31  5.32  6.47  9.35  11.15  13.10  14.85
2023-10-19   7.57  3.18  4.00  5.00  6.17  9.15  10.90  12.95  15.05
2023-10-18   3.15  1.02  1.38  1.84  2.41  4.06   5.20   6.53   8.07
2023-10-17   2.84  0.92  1.24  1.65  2.17  3.68   4.71   5.92   7.46
2023-10-16   5.80  2.30  2.94  3.73  4.67  7.10   8.59  10.25  12.20
2023-10-13   5.81  2.26  2.91  3.71  4.67  7.19   8.78  10.50  12.35
2023-10-12   2.92  1.05  1.37  1.77  2.32  3.72   4.65   5.82   7.22
2023-10-11   2.84  1.11  1.43  1.82  2.30  3.63   4.51   5.54   6.70
2023-10-10   4.81  2.05  2.55  3.19  3.90  5.83   6.99   8.38   9.93


# to_csv

In [129]:
CTB9sp.to_csv("./target_price_set/CTB9sp.csv")
CTB15sp.to_csv("./target_price_set/CTB15sp.csv")
PTB9sp.to_csv("./target_price_set/PTB9sp.csv")
PTB15sp.to_csv("./target_price_set/PTB15sp.csv")